# Step 02
# Training a U-Net model

In [1]:
%matplotlib inline

In [2]:
import sys
import os

import numpy as np
import skimage.io

import tensorflow as tf

import keras.backend
import keras.callbacks
import keras.layers
import keras.models
import keras.optimizers

import utils.model_builder
import utils.data_provider
import utils.metrics
import utils.objectives
import utils.dirtools

# Uncomment the following line if you don't have a GPU
#os.environ['CUDA_VISIBLE_DEVICES'] = ''

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


# Configuration

In [3]:
from config import config_vars

In [4]:
experiment_name = '09'

config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)

data_partitions = utils.dirtools.read_data_partitions(config_vars)

config_vars

{'augment_images': False,
 'batch_size': 10,
 'boundary_boost_factor': 1,
 'boundary_labels_dir': '/data1/image-segmentation/dsb_eval/boundary_labels/',
 'boundary_size': 2,
 'cell_min_size': 16,
 'create_split_files': False,
 'crop_size': 256,
 'csv_log_file': '/data1/image-segmentation/dsb_eval/experiments/09/log.csv',
 'elastic_augmentations': 10,
 'elastic_distortion': 5,
 'elastic_points': 16,
 'epochs': 50,
 'experiment_dir': '/data1/image-segmentation/dsb_eval/experiments/09/out/',
 'labels_out_dir': '/data1/image-segmentation/dsb_eval/experiments/09/out/segm/',
 'learning_rate': 0.0001,
 'max_training_images': 0,
 'min_nucleus_size': 5,
 'model_file': '/data1/image-segmentation/dsb_eval/experiments/09/model.hdf5',
 'normalized_images_dir': '/data1/image-segmentation/dsb_eval/norm_images/',
 'object_dilation': 3,
 'path_files_test': '/data1/image-segmentation/dsb_eval/test.txt',
 'path_files_training': '/data1/image-segmentation/dsb_eval/training.txt',
 'path_files_validation': 

# Initiate data generators

In [5]:
#build session running on GPU 1
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "1"
session = tf.Session(config = configuration)

# apply session
keras.backend.set_session(session)

train_gen = utils.data_provider.random_sample_generator(
    config_vars["normalized_images_dir"],
    config_vars["boundary_labels_dir"],
    data_partitions["training"],
    config_vars["batch_size"],
    config_vars["pixel_depth"],
    config_vars["crop_size"],
    config_vars["crop_size"],
    config_vars["rescale_labels"],
    scales=[1], invert=False
)

val_gen = utils.data_provider.single_data_from_images(
     config_vars["normalized_images_dir"],
     config_vars["boundary_labels_dir"],
     data_partitions["validation"],
     config_vars["val_batch_size"],
     config_vars["pixel_depth"],
     config_vars["crop_size"],
     config_vars["crop_size"],
     config_vars["rescale_labels"]
)

config_vars["val_batch_size"] = 2

# Build model

In [6]:
# build model
model = utils.model_builder.get_model_3_class(config_vars["crop_size"], config_vars["crop_size"], activation=None)
model.summary()

#loss = "categorical_crossentropy"
loss = utils.objectives.weighted_crossentropy

metrics = [keras.metrics.categorical_accuracy, 
           utils.metrics.channel_recall(channel=0, name="background_recall"), 
           utils.metrics.channel_precision(channel=0, name="background_precision"),
           utils.metrics.channel_recall(channel=1, name="interior_recall"), 
           utils.metrics.channel_precision(channel=1, name="interior_precision"),
           utils.metrics.channel_recall(channel=2, name="boundary_recall"), 
           utils.metrics.channel_precision(channel=2, name="boundary_precision"),
          ]

optimizer = keras.optimizers.RMSprop(lr=config_vars["learning_rate"])

model.compile(loss=loss, metrics=metrics, optimizer=optimizer)

# Performance logging
callback_csv = keras.callbacks.CSVLogger(filename=config_vars["csv_log_file"])

callbacks=[callback_csv]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       batch_normalization_1[0][0]      
__________________________________________________________________________________________________
batch_norm

# Training 

In [7]:
# TRAIN
statistics = model.fit_generator(
    generator=train_gen,
    steps_per_epoch=config_vars["steps_per_epoch"],
    epochs=config_vars["epochs"],
    validation_data=val_gen,
    validation_steps=int(len(data_partitions["validation"])/config_vars["val_batch_size"]),
    callbacks=callbacks,
    verbose = 1
)

model.save_weights(config_vars["model_file"])

print('Done! :)')

Epoch 1/50
Training with 160 images.
200/200 [==============================] - 73s 367ms/step - loss: 0.4261 - categorical_accuracy: 0.5273 - background_recall: 0.8667 - background_precision: 0.7280 - interior_recall: 0.4120 - interior_precision: 0.1005 - boundary_recall: 0.7613 - boundary_precision: 0.1668 - val_loss: 2.6915 - val_categorical_accuracy: 0.1888 - val_background_recall: 0.2004 - val_background_precision: 0.4928 - val_interior_recall: 0.0389 - val_interior_precision: 0.0094 - val_boundary_recall: 0.4545 - val_boundary_precision: 0.1975
Epoch 2/50
200/200 [==============================] - 68s 342ms/step - loss: 0.3308 - categorical_accuracy: 0.7740 - background_recall: 0.8991 - background_precision: 0.5786 - interior_recall: 0.5560 - interior_precision: 0.4172 - boundary_recall: 0.8312 - boundary_precision: 0.2403 - val_loss: 1.8959 - val_categorical_accuracy: 0.7036 - val_background_recall: 0.8530 - val_background_precision: 0.8072 - val_interior_recall: 0.0019 - val_in

200/200 [==============================] - 68s 341ms/step - loss: 0.1805 - categorical_accuracy: 0.9309 - background_recall: 0.9437 - background_precision: 0.5076 - interior_recall: 0.7486 - interior_precision: 0.8751 - boundary_recall: 0.9348 - boundary_precision: 0.3114 - val_loss: 2.9401 - val_categorical_accuracy: 0.7478 - val_background_recall: 0.9181 - val_background_precision: 0.8086 - val_interior_recall: 0.0022 - val_interior_precision: 0.0096 - val_boundary_recall: 0.2601 - val_boundary_precision: 0.2093
Epoch 17/50
200/200 [==============================] - 68s 341ms/step - loss: 0.1544 - categorical_accuracy: 0.9339 - background_recall: 0.9462 - background_precision: 0.4588 - interior_recall: 0.7626 - interior_precision: 0.8406 - boundary_recall: 0.9376 - boundary_precision: 0.3058 - val_loss: 2.7514 - val_categorical_accuracy: 0.7196 - val_background_recall: 0.8704 - val_background_precision: 0.8161 - val_interior_recall: 5.3918e-04 - val_interior_precision: 0.0014 - val_b

200/200 [==============================] - 68s 342ms/step - loss: 0.1353 - categorical_accuracy: 0.9376 - background_recall: 0.9576 - background_precision: 0.5183 - interior_recall: 0.8071 - interior_precision: 0.8715 - boundary_recall: 0.9554 - boundary_precision: 0.3479 - val_loss: 3.2068 - val_categorical_accuracy: 0.7221 - val_background_recall: 0.8754 - val_background_precision: 0.8271 - val_interior_recall: 0.0049 - val_interior_precision: 0.0132 - val_boundary_recall: 0.4402 - val_boundary_precision: 0.2216
Epoch 32/50
200/200 [==============================] - 68s 341ms/step - loss: 0.1357 - categorical_accuracy: 0.9415 - background_recall: 0.9561 - background_precision: 0.5026 - interior_recall: 0.8103 - interior_precision: 0.8872 - boundary_recall: 0.9556 - boundary_precision: 0.3621 - val_loss: 3.1099 - val_categorical_accuracy: 0.7238 - val_background_recall: 0.8762 - val_background_precision: 0.8248 - val_interior_recall: 0.0018 - val_interior_precision: 0.0054 - val_bound

200/200 [==============================] - 68s 341ms/step - loss: 0.1017 - categorical_accuracy: 0.9372 - background_recall: 0.9668 - background_precision: 0.5032 - interior_recall: 0.8483 - interior_precision: 0.8515 - boundary_recall: 0.9651 - boundary_precision: 0.3439 - val_loss: 3.2046 - val_categorical_accuracy: 0.7165 - val_background_recall: 0.8687 - val_background_precision: 0.8238 - val_interior_recall: 0.0067 - val_interior_precision: 0.0125 - val_boundary_recall: 0.4477 - val_boundary_precision: 0.2317
Epoch 47/50
200/200 [==============================] - 68s 341ms/step - loss: 0.1079 - categorical_accuracy: 0.9355 - background_recall: 0.9652 - background_precision: 0.5078 - interior_recall: 0.8473 - interior_precision: 0.8777 - boundary_recall: 0.9657 - boundary_precision: 0.3379 - val_loss: 3.1591 - val_categorical_accuracy: 0.7076 - val_background_recall: 0.8578 - val_background_precision: 0.8188 - val_interior_recall: 0.0060 - val_interior_precision: 0.0121 - val_bound